In [1]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [2]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests

In [ ]:
from app.utils.technical_indicator import TechnicalIndicator
indicator = TechnicalIndicator()

band = indicator.cal_bollinger_band()

band = 

In [ ]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time
from datetime import date, timedelta

id = "id1"

auto_trading_stock = AutoTradingBot(id=id, virtual=False)

symbol = '001040'  # CJ
start_date = date(2023, 1, 1)
end_date = date(2024, 1, 1)
initial_capital = 5_000_000
target_trade_value_krw = 1_000_000
buy_trading_logic = ['ema_breakout_trading2']
sell_trading_logic = ['rsi_trading']
interval = 'day'

# 예수금과 상태를 계속 누적 추적할 객체
accumulated_trading_history = {
    'initial_capital': initial_capital,
    'total_quantity': 0,
    'average_price': 0,
    'history': [],
    'buy_dates': [],
    'sell_dates': [],
    'realized_pnl': 0,
    'unrealized_pnl': 0,
    'realized_roi': 0,
    'unrealized_roi': 0,
    'buy_count': 0,
    'sell_count': 0,
}

# 시뮬 날짜 범위
date_range = pd.date_range(start=start_date, end=end_date)

for current_date in date_range:
    print(f"\n📅 {current_date.strftime('%Y-%m-%d')} 시뮬레이션 중")

    try:
        # 하루치 데이터로 시뮬레이션 수행
        daily_result = auto_trading_stock.whole_simulate_trading(
            symbol=symbol,
            start_date=current_date - timedelta(days=100),  # 지표용 데이터 확보
            end_date=current_date,
            target_trade_value_krw=target_trade_value_krw,
            buy_trading_logic=buy_trading_logic,
            sell_trading_logic=sell_trading_logic,
            interval=interval,
            buy_percentage=None,
            ohlc_mode='default',
            initial_capital=accumulated_trading_history['initial_capital']  # 계속 업데이트
        )

        # 누적 자산 정보 갱신
        if daily_result:
            # 거래 내역 붙이기
            accumulated_trading_history['history'].extend(daily_result['history'])
            accumulated_trading_history['buy_dates'].extend(daily_result['buy_dates'])
            accumulated_trading_history['sell_dates'].extend(daily_result['sell_dates'])
            accumulated_trading_history['buy_count'] += daily_result['buy_count']
            accumulated_trading_history['sell_count'] += daily_result['sell_count']
            accumulated_trading_history['realized_pnl'] += daily_result['realized_pnl']
            accumulated_trading_history['unrealized_pnl'] = daily_result['unrealized_pnl']
            accumulated_trading_history['realized_roi'] = daily_result['realized_roi']
            accumulated_trading_history['unrealized_roi'] = daily_result['unrealized_roi']
            accumulated_trading_history['initial_capital'] = daily_result['initial_capital']  # 예수금 반영
            accumulated_trading_history['total_quantity'] = daily_result['total_quantity']
            accumulated_trading_history['average_price'] = daily_result['average_price']

    except Exception as e:
        print(f"⚠️ {current_date.strftime('%Y-%m-%d')} 시뮬레이션 실패: {str(e)}")

# ✅ 결과 요약 출력
print("\n===== ✅ 누적 시뮬레이션 결과 요약 =====")
print(f"총 매수 횟수: {accumulated_trading_history['buy_count']}")
print(f"총 매도 횟수: {accumulated_trading_history['sell_count']}")
print(f"총 실현 손익: {accumulated_trading_history['realized_pnl']:.0f} KRW")
print(f"총 미실현 손익: {accumulated_trading_history['unrealized_pnl']:.0f} KRW")
print(f"최종 예수금: {accumulated_trading_history['initial_capital']:.0f} KRW")
print(f"보유 수량: {accumulated_trading_history['total_quantity']}")
print(f"평균 매수가: {accumulated_trading_history['average_price']:.2f} KRW")




실전투자 API 객체가 성공적으로 생성되었습니다.

📅 2023-01-01 시뮬레이션 중
⚠️ 2023-01-01 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-02 시뮬레이션 중
⚠️ 2023-01-02 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-03 시뮬레이션 중
⚠️ 2023-01-03 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-04 시뮬레이션 중
⚠️ 2023-01-04 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-05 시뮬레이션 중
⚠️ 2023-01-05 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-06 시뮬레이션 중
⚠️ 2023-01-06 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-07 시뮬레이션 중
⚠️ 2023-01-07 시뮬레이션 실패: AutoTradingBot.whole_simulate_trading() missing 1 required positional argument: 'df'

📅 2023-01-08 시뮬레이션 중
⚠️ 2023-01-08 시뮬레이션 실패: AutoTradi

In [4]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time

id = 'bnuazz15'
trading_bot_name = 'test_bot'
auto_trading_stock = AutoTradingBot(id=id, virtual=True)

symbol = '207940'
symbol_name = '삼성바이오로직스'
start_date = date(2023, 3, 1)
end_date = date(2025, 4, 3)
interval = 'day'
max_allocation = 0.01
target_trade_value_krw = 1000000  # 매수 목표 거래 금액
buy_trading_logic = ['check_wick']
sell_trading_logic = ['check_wick']

auto_trading_stock.trade(
    trading_bot_name=trading_bot_name,
    buy_trading_logic=buy_trading_logic,
    sell_trading_logic=sell_trading_logic,
    symbol=symbol,
    symbol_name=symbol_name,
    start_date=start_date,
    end_date=end_date,
    target_trade_value_krw=target_trade_value_krw,
    interval=interval,
    max_allocation = max_allocation
)



모의투자 API 객체가 성공적으로 생성되었습니다.
시간: 2025-04-03 00:00:00+09:00, open_price: 1035000 KRW, low_price: 1035000 KRW, high_price: 1127000 KRW, close_price: 1113000 KRW, 볼린저 밴드 정보: 상단: 1116942 KRW, 중단: 1070850 KRW, 하단: 1024758 KRW
윗꼬리 = 14000.0, 아랫꼬리 = 0.0, body = 78000.0
아랫꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님
윗꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님, 종가가 시가보다 낮지 않음, 볼린저 밴드 상단 근처가 아님
check_wick 로직 buy_signal = False
시간: 2025-04-03 00:00:00+09:00, open_price: 1035000 KRW, low_price: 1035000 KRW, high_price: 1127000 KRW, close_price: 1113000 KRW, 볼린저 밴드 정보: 상단: 1116942 KRW, 중단: 1070850 KRW, 하단: 1024758 KRW
윗꼬리 = 14000.0, 아랫꼬리 = 0.0, body = 78000.0
아랫꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님
윗꼬리 감지 실패: 시간: 2025-04-03 00:00:00+09:00, 사유: 아랫꼬리가 윗꼬리보다 짦음, 하락 추세가 아님, 종가가 시가보다 낮지 않음, 볼린저 밴드 상단 근처가 아님
check_wick 로직 sell_signal = False
마지막 직전 봉 : 78000.0. 양봉 : True, 음봉 : False


In [3]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time
from app.scheduler.auto_trading_scheduler import scheduled_trading_schedulerbot_task, scheduled_trading_bnuazz15_task, scheduled_trading


scheduled_trading(id = 'bnuazz15', virtual = True, trading_bot_name = 'bnuazz15bot')



모의투자 API 객체가 성공적으로 생성되었습니다.
- buy_trading_logic: ['ema_breakout_trading2', 'ema_breakout_trading', 'sma_breakout_trading', 'macd_trading', 'trend_entry_trading', 'bottom_rebound_trading', 'bollinger_band_trading'], sell_trading_logic : ['rsi_trading', 'macd_trading', 'top_reversal_sell_trading', 'downtrend_sell_trading', 'bollinger_band_trading']
[04/07 14:03:44] WARNING: API 호출 횟수를 초과하여 호출 유량 획득까지 대기합니다.
🗑️ 삭제된 종목: 025900
🗑️ 삭제된 종목: 025980
🗑️ 삭제된 종목: 114090
🗑️ 삭제된 종목: 141080
🗑️ 삭제된 종목: 225570
🗑️ 삭제된 종목: 241710
🗑️ 삭제된 종목: 263750
🗑️ 삭제된 종목: 326030
✅ 트랜잭션 성공: 1744002225858
[💾 잔고 저장] 025980
✅ 트랜잭션 성공: 1744002226513
[💾 잔고 저장] 114090
✅ 트랜잭션 성공: 1744002227170
[💾 잔고 저장] 241710
✅ 트랜잭션 성공: 1744002227801
[💾 잔고 저장] 263750


In [6]:
from app.utils.auto_trading_bot import AutoTradingBot
from pykis import KisBalance

auto_trading_stock = AutoTradingBot(id = 'bnuazz15bot_real', virtual= False)
trading_bot_name = 'bnuazz15bot_real'
account = auto_trading_stock.kis.account()

balance: KisBalance = account.balance()


실전투자 API 객체가 성공적으로 생성되었습니다.


In [13]:
balance

KisIntegrationBalance(
    account_number=KisAccountNumber('67737279-01'),
    deposits={
        'KRW': KisDomesticDeposit(account_number=KisAccountNumber('67737279-01'), currency='KRW', amount=5005521, exchange_rate=1)
    },
    stocks=[
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='001680', qty=42, price=23800, amount=999600, profit=-4200, profit_rate=-0.41841004184100418410041841),
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='006260', qty=8, price=125000, amount=1000000, profit=-17600, profit_rate=-1.729559748427672955974842767),
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='108860', qty=69, price=14200, amount=979800, profit=-20010, profit_rate=-2.001380262249827467218771567),
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='280360', qty=8, price=119900, amount=

In [5]:
from datetime import date
from pykis import KisOrderProfits

def get_total_profits_from(start_date: date):
    return account.profits(start=start_date, end=date.today())

profits = get_total_profits_from(date(2023, 8, 1))
print(repr(profits))

KisIntegrationOrderProfits(
    account_number=KisAccountNumber('64156776-01'),
    buy_amount=1353000,
    sell_amount=1530350,
    profit=177350,
    orders=[
        KisDomesticOrderProfit(time_kst='2025-04-17T00:00:00+09:00', market='KRX', symbol='042000', name='카페24', buy_price=54600, sell_price=59278, qty=9, profit=42100, profit_rate=8.567358567358567358567358567),
        KisDomesticOrderProfit(time_kst='2025-04-16T00:00:00+09:00', market='KRX', symbol='298380', name='에이비엘바이오', buy_price=50700, sell_price=63706, qty=9, profit=117050, profit_rate=25.65198334429103659872890642),
        KisDomesticOrderProfit(time_kst='2025-04-15T00:00:00+09:00', market='KRX', symbol='010620', name='HD현대미포', buy_price=115500, sell_price=132900, qty=1, profit=17400, profit_rate=15.06493506493506493506493506),
        KisDomesticOrderProfit(time_kst='2025-04-15T00:00:00+09:00', market='KRX', symbol='010140', name='삼성중공업', buy_price=14490, sell_price=14530, qty=20, profit=800, profit_rate=0.276052449

In [2]:
from app.scheduler.auto_trading_scheduler import scheduled_single_buy_task

scheduled_single_buy_task()

실전투자 API 객체가 성공적으로 생성되었습니다.
[2025-04-22 14:10:11.315139] 자동 매수 실행: 종목 300720, 수량 1주
주문 처리 중 오류 발생: KIS API 요청에 실패했습니다.
(RT_CD: 7, MSG_CD: APBN0678) TTTC8434R
비밀번호 1회 오류입니다.(5회 오류시 거래불가!!!)

[  Request  ]: GET https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/trading/inquire-balance
Headers: {'User-Agent': 'PyKis/2.1.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'tr_id': 'TTTC8434R', 'appkey': '***', 'appsecret': '***', 'Authorization': 'Bearer ***'}
Params: {'AFHR_FLPR_YN': ['N'], 'INQR_DVSN': ['02'], 'UNPR_DVSN': ['01'], 'FUND_STTL_ICLD_YN': ['Y'], 'FNCG_AMT_AUTO_RDPT_YN': ['N'], 'PRCS_DVSN': ['00'], 'CANO': ['64927922'], 'ACNT_PRDT_CD': ['01']}
Body: [EMPTY BODY]
 예수금 : None, 
메시지가 성공적으로 전송되었습니다.


In [3]:
from app.utils.dynamodb.model.stock_symbol_model import StockSymbol 
    
result = list(StockSymbol.scan(
        filter_condition=((StockSymbol.type == 'kosdaq150'))
    )) #scan은 랜덤, 정렬 불가 -> 종목 순서 기준은 추후 검토
    
sorted_result = sorted(
        result,
        key=lambda x: (x.type, x.symbol_name)
    )

In [ ]:
sorted_result

In [ ]:
#주문가능금액 API
import requests
from app.utils.auto_trading_bot import AutoTradingBot

id = "bnuazz15bot"
symbol = '005930'
auto_trading_stock = AutoTradingBot(id=id, virtual=True)

psbl_order_info = auto_trading_stock.inquire_psbl_order(symbol)
#print(auto_trading_stock.account)
#print(auto_trading_stock.kis.token)

실전투자 API 객체가 성공적으로 생성되었습니다.


In [3]:
psbl_order_info

{'output': {'ord_psbl_cash': '1202269',
  'ord_psbl_sbst': '2840830',
  'ruse_psbl_amt': '1457505',
  'fund_rpch_chgs': '0',
  'psbl_qty_calc_unpr': '72400',
  'nrcvb_buy_amt': '2626724',
  'nrcvb_buy_qty': '36',
  'max_buy_amt': '2626724',
  'max_buy_qty': '36',
  'cma_evlu_amt': '0',
  'ovrs_re_use_amt_wcrc': '0',
  'ord_psbl_frcr_amt_wcrc': '0'},
 'rt_cd': '0',
 'msg_cd': 'KIOK0510',
 'msg1': '조회가 완료되었습니다                                                           '}